In [2]:
import requests
import json
import pandas as pd
import numpy as np
import pickle
from flask import jsonify

In [23]:
url = 'http://3.99.217.110:5000/predict'

json_data =[{
    "Credit_History" : 1,
    "ApplicantIncome" : 90,
    "CoapplicantIncome" : 90,
    "LoanAmount" : 100,
    "Education" : 'Graduate',
    "Self_Employed" : 'No',
    "Gender" : 'Female',
    "Married" : 'No',
    "Dependants" : 2,
    "Property_Area" : 'Urban',
    "Loan_Amount_Term" : 360
    }]

r = requests.post(url, json=json_data)

print(r.json())


{'Request': 'Approved'}


In [57]:
query_ = pd.DataFrame(json_data)
model_columns = ['Credit_History', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
                 'Education', 'Self_Employed', 'Gender', 'Married', 'Dependents', 
                 'Property_Area','Loan_Amount_Term']
query = query_.reindex(columns = model_columns, fill_value=0)
         
with open('./loan_predict.pickle', 'rb') as f:
    model = pickle.load(f)   
         
query['CoapplicantIncome'] = query.CoapplicantIncome.astype('int')
query['ApplicantIncome'] = query.ApplicantIncome.astype('int')
query['Total_Income'] = query['ApplicantIncome'] + query['CoapplicantIncome']
query['Total_Income'] = np.log(query['Total_Income'])
query['LoanAmount'] = np.log(query['LoanAmount'])
query['Education'] = query.Education.map({'Graduate':0, 'Not Graduate':1})
query['Self_Employed'] = query.Self_Employed.map({'Yes':0, 'No':1})
query['Property_Area'] = query.Property_Area.map({'Urban':0, 'Rural':1, 'Semiurban':2})
query['Credit_History'] = query.Credit_History.astype('int64')
query = query[['Credit_History', 'Self_Employed', 'Total_Income', 
               'LoanAmount' , 'Property_Area', 'Education']]
            
prediction = list(model.predict(query))
if prediction[0] == False:
    prediction = "Rejected"
if prediction[0] == True:
    prediction = "Approved"

print("Request", prediction)

Request Approved
